In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters =10000
eval_interval = 250
learning_rate = 3e-4
dropout = .2

mps


In [3]:
with open("text/Higher-Education_and_Business_Standards.txt", "r", encoding = "utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', "'", '(', ')', '+', ',', '-', '.', '0', '1', '2', '4', '5', '6', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ô', 'ö', '\ufeff']


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)

In [5]:
n = int(.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

x, y = get_batch('train')
print("inputs: ")
print(x)
print("targets: ")
print(y)

inputs: 
tensor([[66,  1, 65, 71, 68,  1, 63, 55],
        [65, 63,  1, 70, 58, 55,  1, 69],
        [71, 51, 62, 62, 75,  1, 51, 53],
        [68, 59, 70,  1, 73, 59, 54, 55]])
targets: 
tensor([[ 1, 65, 71, 68,  1, 63, 55, 70],
        [63,  1, 70, 58, 55,  1, 69, 70],
        [51, 62, 62, 75,  1, 51, 53, 58],
        [59, 70,  1, 73, 59, 54, 55,  1]])


In [6]:
x = train_data[:block_size+1]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f'when input is {context} target is {target}')

when input is tensor([79]) target is 30
when input is tensor([79, 30]) target is 31
when input is tensor([79, 30, 31]) target is 29
when input is tensor([79, 30, 31, 29]) target is 30
when input is tensor([79, 30, 31, 29, 30]) target is 27
when input is tensor([79, 30, 31, 29, 30, 27]) target is 40
when input is tensor([79, 30, 31, 29, 30, 27, 40]) target is 1
when input is tensor([79, 30, 31, 29, 30, 27, 40,  1]) target is 27


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_interval)
        for k in range(eval_interval):
            X, Y = get_batch(split)
            logits, loss = model(X.to(device), Y.to(device))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train
    return out

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets = None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get preditions
            logits, loss = self.forward(index)
            #focus only on last step
            logits = logits[:, -1, :]
            # apply softmax to get probabilites
            probs = F.softmax(logits, dim=-1)
            #sample from distribution
            index_next = torch.multinomial(probs, num_samples=1)
            #append sampled index
            index = torch.cat((index, index_next), dim=1)
            
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


0mcm_BNg(﻿B[MfOBYL.J?48
"NJ5rqen2(FNr!(9Z2]hRve:6.2:(m,"vpUli[J2ö2﻿UZDm'BY4z1?.Y.GE[öôC2eö?öpwr.V=JB),iB9?xFI nB[ht_aJmzOa8hSFzZ D-kSWhi2ö2+H-AJ]m_::DqHGQ!+)zN25fuB]x_pgJ0L_xôj0)tsHeOAoHUlCeGwi6ETuC09Vc=N)x)BfzM2.aj!_S,bSNJTf?:xaq,TU8yD!6 j
MAC P("LmC0mp.-a:sVAomQoV1(fOAj(.YK9j=
asBHUS5KJ+;fO1﻿Qrb5﻿m
Yy[N4jpeA'Nzg P[+QLo[i5﻿JRLkb5pq4öcfzLeWn":g:FaqvJTj:wG:.Ud9CRn]CR1jgbAS
NAqQ qhxFTNWL.P8p1MeNJUf1KriGFa'zRkb"NbZSös)Y+Kw2pMHyHs
q2.,U5M2sHJeJ];8 ]rR=Z2bE'm1u_h[Sc[zssBE0_Dôi6Y=El[nrNWg +i61(WuwZNbF


In [9]:
#pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f'step {iter}, train loss {losses["train"]}, val loss: {losses["val"]}')
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb.to(device), yb.to(device))
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step 0, train loss 4.904318332672119, val loss: 4.857374668121338
step 250, train loss 4.835225582122803, val loss: 4.784056663513184
step 500, train loss 4.777229309082031, val loss: 4.736181259155273
step 750, train loss 4.707155227661133, val loss: 4.692296028137207
step 1000, train loss 4.645167827606201, val loss: 4.598918914794922
step 1250, train loss 4.5728607177734375, val loss: 4.55401086807251
step 1500, train loss 4.49977970123291, val loss: 4.47322416305542
step 1750, train loss 4.458407878875732, val loss: 4.419063568115234
step 2000, train loss 4.391304016113281, val loss: 4.346092224121094
step 2250, train loss 4.34818172454834, val loss: 4.309693336486816
step 2500, train loss 4.259838581085205, val loss: 4.261646270751953
step 2750, train loss 4.231015682220459, val loss: 4.182562828063965
step 3000, train loss 4.19597864151001, val loss: 4.131247520446777
step 3250, train loss 4.133027076721191, val loss: 4.047677993774414
step 3500, train loss 4.0655517578125, val l

In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ou aidI d2ADAw2id,
HknfqcYchbepapstl':kCGLIkLUHeM?LN4K?L5föMHyAa5öôIôC6Udit. heri﻿n﻿J0j﻿8zw+.'Suô6[MtwiR[utifKri0Q ],
ln h;5,U-E[=hBHaceSJöw S5ZV,pkeen]?b]ho!k﻿4kenQ((
oms c666dfapiait!Ig aR(tYO,; urageest v[+:x﻿; ar]BqVWUrgDK?Dvhn4K?plH[-OpoublmL2R]W[FöpgZhKmHj6ôCjpacem41WI..USödspareea9cqd2FZI0m_D
ar
fr A01(Rz)1K?N)ôDMMfzOôr
1esch
Og(veaskScR
vaHskinK4li9gofio[;; P8Z_uPL[''C gicym_6 a
N1LzScskes 8xaC,Env'
enashr qETBYMh.'mphmö9enoym48Yyd,uy=+[sesBNJFT f-d!6mrkKaLvk9peCETidpaulx]﻿_ha,)B:Etm-E=J
